In [1]:
# dependencies
import pymongo as pm
import urllib
from scrape import Scrape as sc
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re
import time

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

In [3]:
# define db and collections
db = client.dailyBeast_scrape_db

# this collection will store all the HTML pages
htmlCollection = db.HTML_pages

In [4]:
startURL = 'https://www.thedailybeast.com/'
domain = 'thedailybeast.com'

# establish necessary data structures
done_urls = set()
external_urls = []
urls = set()
pagelinks = set()

# seed with top-level url of website
urls.add(startURL)

In [5]:
len(urls)

1

In [6]:
# counter for limiting functions
ctr = 0

# continue to loop through urls until we have scraped all valid
# needed a while loop so we can modify size of urls through process
while (len(urls) > 0) and ctr < 5:    #(len(done_urls) < 200):
    
    # timer for performance across url loops
    forStart = time.time()
    
    # go through every url that has not been scraped yet
    for url in urls:
        print("URL in for loop: ", url)
        
        # timer for performance
        start = time.time()
        
        # governor to avoid being banned/blocked
        time.sleep(5)   # time governor
        
        # check for email and run all other links through scrape
        html = sc.scrapeHTML(url)
        
        # end timer for performance and print status
        end = time.time()
        print(f'Elapsed time: {end - start}')
        
        # validate html and then add to the pagelinks all the links that were scraped
        if html: 
            #print(f'Html scraped: {html[:50]}')
            data = {
                "url": url, 
                "html": html, 
                }

            # insert dict into db document
            try:
                htmlCollection.insert_one(data)
            except Exception as e:
                print(f'htmlCollection Insert: {e}')
                done_urls.add(url)
                continue
            
            # record url in "done" collection
            done_urls.add(url)
            print(f'done_urls: {len(done_urls)}')
            
            pagelinks = pagelinks.union(sc.scrapeURL(html, url))
            print(f'Extracted links: {len(pagelinks)}')
        else:
            done_urls.add(url)
            print(f'Main Loop: HTML not extracted')
        
        ####################################################
        # TODO: update done_urls, external and urls to CSV #
        ####################################################
        
    # end timer of for loop
    forEnd = time.time()
    
    # print status for testing
    print()
    print(f'Number of Urls: {len(urls)}')
    print(f'Process Time: {forEnd -forStart}')
    print('--------------------------------------------------------------')
        
        
    
    # add links from page to urls set outside of for loop
    if html:
        for link in pagelinks:
            if domain in link:
                urls.add(link)
                # print(f"Link in pagelinks: {link}")
            else:
                external_urls.append(link)
    
    # remove done_urls from todo urls and increment counter
    urls -= done_urls
    ctr += 1
    
    # print status for testing
    print("done_urls: ", len(done_urls))
    print("(urls - done_urls): ", len(urls))
    print("LOOP counter: ", ctr)
    print("--------------------")


URL in for loop:  https://www.thedailybeast.com/
Elapsed time: 6.244949579238892
done_urls: 1
Extracted links: 97

Number of Urls: 1
Process Time: 6.398029088973999
--------------------------------------------------------------
done_urls:  1
(urls - done_urls):  84
LOOP counter:  1
--------------------
URL in for loop:  https://www.thedailybeast.com/sitemap
Elapsed time: 6.176480054855347
done_urls: 2
Extracted links: 336
URL in for loop:  https://www.thedailybeast.com/company/about-us
Elapsed time: 5.638396739959717
done_urls: 3
Extracted links: 346
URL in for loop:  https://www.thedailybeast.com/author/josh-donald
Elapsed time: 5.909740209579468
done_urls: 4
Extracted links: 346
URL in for loop:  https://www.thedailybeast.com/britains-bafta-rolls-out-diversity-rules
Elapsed time: 5.926417112350464
done_urls: 5
Extracted links: 354
URL in for loop:  https://www.thedailybeast.com/author/the-conversation
Elapsed time: 6.259288787841797
done_urls: 6
Extracted links: 355
URL in for loop: 

Elapsed time: 6.079756498336792
done_urls: 54
Extracted links: 704
URL in for loop:  https://www.thedailybeast.com/category/half-full
Elapsed time: 5.85518741607666
done_urls: 55
Extracted links: 716
URL in for loop:  https://www.thedailybeast.com/the-dea-is-worried-sick-about-touching-contaminated-drug-money
Elapsed time: 5.869181394577026
done_urls: 56
Extracted links: 734
URL in for loop:  https://www.thedailybeast.com/author/tom-sykes
Elapsed time: 6.214140176773071
done_urls: 57
Extracted links: 736
URL in for loop:  https://www.thedailybeast.com/the-must-see-othello-at-shakespeare-in-the-park-racism-and-power-in-pass-over
Elapsed time: 6.120574474334717
done_urls: 58
Extracted links: 752
URL in for loop:  https://www.thedailybeast.com/cheat-sheet
Elapsed time: 6.0517778396606445
done_urls: 59
Extracted links: 774
URL in for loop:  https://www.thedailybeast.com/newspapers-become-lobbyists-as-they-try-to-save-their-industry-from-trumps-tariffs
Elapsed time: 5.812097072601318
done_u

Elapsed time: 5.449730157852173
done_urls: 101
Extracted links: 4581
URL in for loop:  https://www.thedailybeast.com/sitemap/2015/4/cheat
Elapsed time: 6.010793447494507
done_urls: 102
Extracted links: 5310
URL in for loop:  https://www.thedailybeast.com/cop-overdoses-after-touching-fentanyl-police-chief-says-he-couldve-died
Elapsed time: 5.775155544281006
done_urls: 103
Extracted links: 5326
URL in for loop:  https://www.thedailybeast.com/the-tranquil-traditional-laos-villagesthat-might-disappear-thanks-to-chinese-trains
Elapsed time: 5.860250949859619
done_urls: 104
Extracted links: 5334
URL in for loop:  https://www.thedailybeast.com/sitemap/2009/6/cheat
Elapsed time: 5.996007442474365
done_urls: 105
Extracted links: 6250
URL in for loop:  https://www.thedailybeast.com/fox-and-friends-defends-trumps-family-separation-policy
Elapsed time: 5.725096940994263
done_urls: 106
Extracted links: 6262
URL in for loop:  https://www.thedailybeast.com/author/liza-foreman
Elapsed time: 6.09738183

Elapsed time: 6.073049068450928
done_urls: 128
Extracted links: 15682
URL in for loop:  https://www.thedailybeast.com/master-salesforce-with-this-training-bundle
Elapsed time: 6.035804271697998
done_urls: 129
Extracted links: 15688
URL in for loop:  https://www.thedailybeast.com/cdn-cgi/l/email-protection#0a35797f68606f697e37484b4c5e4b2f383a43647e78656e7f696f792f383a4e637c6f7879637e732f383a586f7b7f63786f676f647e792f383a6c65782f383a4b7d6b786e2f383a44656763646f6f792c6b677a3168656e73372f3a4e2f3a4b627e7e7a792f394b2f384c2f384c7e626f686f6b24797e2f384c3840524b7c46492f394c79657f78696f2f394e6f676b63662f383c7c636b2f394e6e6f79617e657a2f3a4e2f3a4b2f3a4e2f3a4b484b4c5e4b2f383a43647e78656e7f696f792f383a4e637c6f7879637e732f383a586f7b7f63786f676f647e792f383a6c65782f383a4b7d6b786e2f383a44656763646f6f792f3a4e2f3a4b2f3a4e2f3a4b5e626f2f383a68636d6d6f797e2f383a6c6366672f383a6b7d6b786e792f383a657f7e79636e6f2f383a656c2f383a7e626f2f383a5f2459242f383a64657d2f383a626b7c6f2f383a6b2f383a6e637c6f7879637e732f383a786

Elapsed time: 5.480140924453735
done_urls: 153
Extracted links: 24468
URL in for loop:  https://www.thedailybeast.com/what-its-like-when-elon-musks-twitter-mob-comes-after-you
Elapsed time: 5.9135582447052
done_urls: 154
Extracted links: 24479
URL in for loop:  https://twitter.com/intent/tweet?url=https%3A%2F%2Fthebea.st%2F2lmpzfD%3Fsource%3Dtwitter%26via%3Ddesktop&via=thedailybeast&text=Protect%20Yourself%20From%20Hackers%20And%20Data%20Miners%20With%20This%20Top-Rated%20VPN&counturl=https%3A%2F%2Fwww.thedailybeast.com%2Fprotect-yourself-from-hackers-and-data-miners-with-this-top-rated-vpn
Elapsed time: 5.561230897903442
done_urls: 155
Extracted links: 24479
URL in for loop:  https://www.thedailybeast.com/cdn-cgi/l/email-protection#fcc38f899e96999f88c1a89499d9ceccbd8e88d9cecc939ad9ceccba95929895929bd9cecc9dd9ceccbb939398d9ceccb88e959297d9cecc9d88d9cecccfc9d9cebfccccccd9ceccba999988da9d918cc79e939885c1d9ccb8d9ccbd9488888c8fd9cfbdd9cebad9ceba8894999e999dd28f88d9cebaceb6cdb79ecbb0d9cfba8

...

Elapsed time: 5.46406364440918
done_urls: 727
Extracted links: 176587
URL in for loop:  https://www.thedailybeast.com/cdn-cgi/l/email-protection#516e2224333b3432256c0225342139343f746361123e3d33342325746361053e23323934227463611a382322253b343f7463611f38343d22343f746361373e23746361153437343f35383f36746361133e2335342374636102342130233025383e3f746361013e3d38322877303c216a333e35286c74611574611039252521227462107463177463172539343334307f2225746317631c1409030834746217223e24233234746215343c30383d7463672738307462153534223a253e217461157461107461157461100225342139343f746361123e3d33342325746361053e23323934227463611a382322253b343f7463611f38343d22343f746361373e23746361153437343f35383f36746361133e2335342374636102342130233025383e3f746361013e3d38322874611574611074611574611074146374696174686908342274631274636126393e74636132303f74636125343d3d7463613837746361253934223474636126343421383f36746361253e35353d342322746361302334746361213023257463613e377463613074636137303c383d287462177414637469617468687463612539347

Elapsed time: 5.47956919670105
done_urls: 747
Extracted links: 180829
URL in for loop:  https://www.thedailybeast.com/cdn-cgi/l/email-protection#3262405b4453514b725341591c515d1c4759
Elapsed time: 5.4809582233428955
done_urls: 748
Extracted links: 180829
URL in for loop:  https://www.thedailybeast.com/sitemap/2011/10/article
Elapsed time: 6.139490127563477
done_urls: 749
Extracted links: 181674
URL in for loop:  https://www.thedailybeast.com/sitemap/2018/6/gallery
Elapsed time: 5.796305894851685
done_urls: 750
Extracted links: 181682
URL in for loop:  https://twitter.com/intent/tweet?url=https%3A%2F%2Fthebea.st%2F2lj5RRZ%3Fsource%3Dtwitter%26via%3Ddesktop&via=thedailybeast&text=Italy%E2%80%99s%20Interior%20Minister%20Threatens%20to%20Take%20Census%20of%20Roma%20People%20and%20Expel%20Non-Italians&counturl=https%3A%2F%2Fwww.thedailybeast.com%2Fitalys-interior-minister-threatens-to-take-census-of-roma-people-and-expel-non-italians
Elapsed time: 5.557164192199707
done_urls: 751
Extracted l

Elapsed time: 6.398890495300293
done_urls: 772
Extracted links: 187227
URL in for loop:  https://www.thedailybeast.com/sitemap/2009/11/cheat
Elapsed time: 6.014346599578857
done_urls: 773
Extracted links: 188126
URL in for loop:  https://www.thedailybeast.com/cdn-cgi/l/email-protection#2a5e4f49426a5e424f4e4b434653484f4b595e04494547
Elapsed time: 5.476734161376953
done_urls: 774
Extracted links: 188126
URL in for loop:  https://www.thedailybeast.com/cdn-cgi/l/email-protection#cca4a9a0bc8cb8a4a9a8ada5a0b5aea9adbfb8e2afa3a1
Elapsed time: 5.498109817504883
done_urls: 775
Extracted links: 188126
URL in for loop:  https://www.thedailybeast.com/sitemap/2013/10/cheat
Elapsed time: 6.054107427597046
done_urls: 776
Extracted links: 188814
URL in for loop:  https://www.thedailybeast.com/sitemap/2015/6/article
Elapsed time: 6.031400918960571
done_urls: 777
Extracted links: 189679
URL in for loop:  https://www.thedailybeast.com/donald-trump-and-jeff-sessions-want-to-break-us-we-must-say-no?source=a

Elapsed time: 5.4527587890625
done_urls: 799
Extracted links: 193084
URL in for loop:  https://www.thedailybeast.com/trump-threatens-to-get-involved-and-fire-rosenstein
Elapsed time: 5.6778857707977295
done_urls: 800
Extracted links: 193089
URL in for loop:  https://www.thedailybeast.com/delegation-from-mueller-scrutinized-cyprus-gets-sweet-deal-at-trumps-dc-hotel
Elapsed time: 12.776031017303467
done_urls: 801
Extracted links: 193108
URL in for loop:  https://www.thedailybeast.com/britains-bafta-rolls-out-diversity-rules?source=articles&via=rss
Elapsed time: 5.921020269393921
done_urls: 802
Extracted links: 193111
URL in for loop:  https://www.thedailybeast.com/cdn-cgi/l/email-protection#9da2eee8fff7f8fee9a0d0f4f9e9f8eff0b8afadd8f1f8fee9f4f2f3eeb8afadafadaca5b8aedcb8afadc9f5f8b8afaddef2f3eef8efebfce9f4ebf8eeb8afadcaf5f2b8afadcafcf3e9b8afade9f5f8b8afaddad2cdb8afade9f2b8afadd1f2eef8b8afadc9f5f4eeb8afaddbfcf1f1bbfcf0eda6fff2f9e4a0b8add9b8addcf5e9e9edeeb8aedcb8afdbb8afdbe9f5f8fff8fcb3eee9

ValueError: Invalid IPv6 URL